In [1]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
dataframe = pd.read_csv("data.csv")
text = list(dataframe.values)
joined_text =' '.join(str(v) for v in text)
with open("joined_text.txt", "w", encoding="utf-8") as f:
    f.write(joined_text)

In [3]:
partial_text = joined_text[:205000]

In [4]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text)

In [5]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [6]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])

    next_word.append(tokens[i + n_words])

In [7]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)  # for each sample, n input words and then a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)  # for each sample a boolean for each possible next word

In [8]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [10]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X, y, batch_size=128, epochs=30, shuffle=True).history

Epoch 1/30
288/288 [==============================] - 15s 24ms/step - loss: 8.0534 - accuracy: 0.0275
Epoch 2/30
288/288 [==============================] - 7s 23ms/step - loss: 7.9107 - accuracy: 0.0282
Epoch 3/30
288/288 [==============================] - 6s 23ms/step - loss: 7.8446 - accuracy: 0.0300
Epoch 4/30
288/288 [==============================] - 7s 24ms/step - loss: 7.7324 - accuracy: 0.0329
Epoch 5/30
288/288 [==============================] - 7s 23ms/step - loss: 7.5649 - accuracy: 0.0358
Epoch 6/30
288/288 [==============================] - 7s 24ms/step - loss: 7.3568 - accuracy: 0.0401
Epoch 7/30
288/288 [==============================] - 7s 24ms/step - loss: 7.1118 - accuracy: 0.0509
Epoch 8/30
288/288 [==============================] - 7s 24ms/step - loss: 6.8184 - accuracy: 0.0697
Epoch 9/30
288/288 [==============================] - 6s 22ms/step - loss: 6.4939 - accuracy: 0.0914
Epoch 10/30
288/288 [==============================] - 7s 24ms/step - loss: 6.1588 - accur

In [11]:
model.save("text_gen_model2.h5")
with open("history2.p", "wb") as f:
    pickle.dump(history, f)

In [12]:
model = load_model("text_gen_model2.h5")
history = pickle.load(open("history2.p", "rb"))

In [13]:
def predict_next_word(input_text, n_best):

    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1

    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [14]:
possible = predict_next_word("نظام جمهوری اسلامی", 30)
for idx in possible:
    print(unique_tokens[idx])

1/1 [==============================] - 1s 635ms/step
میکنه
خب
دیدم
اسم
چرا
وسط
توی
می
باز
تو
باید
کنار
میکنن
که
رو
ما
اینا
فکر
خوبه
ها
خودش
میگه
ای
واقعا
کسی
تا
دارم
خیلی
آدم
هم


In [15]:
def generate_text(input_text, n_words, creativity):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence))[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [24]:
generate_text("نظام جمهوری اسلامی", 1, 10)

1/1 [==============================] - 0s 20ms/step


'نظام جمهوری اسلامی باید'

In [25]:
generate_text("من میخوام برم مشهد", 30, 100)

1/1 [==============================] - 0s 19ms/step


'من میخوام برم مشهد امشب الان شد بر دارن کجا اسب اشتباه آمریکا تشکیل_شورای_انقلاب اشتباه تو دارن ندارد میکنه شبیه اونم جهان هنوز وقتی بگیرن خمینی ملت درد الان زیر دوست دارن پیش واقعا'